# Parte 1 / Creacion de RAG

### Instalaciones

In [1]:
!pip install chromadb gdown sentence-transformers scikit-learn pdfplumber requests beautifulsoup4 selenium rdflib gensim  langdetect pytextrank FlagEmbedding  llama-index-embeddings-huggingface==0.1.1 sentence-transformers==2.3.1 pypdf==4.0.1 langchain==0.1.7 python-decouple==3.8 llm-templates llama-index-readers-file rank_bm25 unidecode nltk deep_translator> /dev/null 2>&1
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb > /dev/null 2>&1
!dpkg -i google-chrome-stable_current_amd64.deb > /dev/null 2>&1
!apt-get -f install -y > /dev/null 2>&1
!wget -q https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.87/linux64/chromedriver-linux64.zip > /dev/null 2>&1
!unzip -q chromedriver-linux64.zip -d /usr/bin/ > /dev/null 2>&1


!python -m spacy download es_core_news_sm
!pip install llama-index-llms-ollama
!pip install wikipedia
!pip install litellm[proxy]
!pip install llama-index
!pip install rank-bm25
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 69.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.68.post1
    Uninstalling llama-index-core-0.10.68.post1:
      Successfully uninstalled llama-index-core-0.10.68.post1
ERROR: pip's dependency resolver does not currently take int

### Librerias

In [2]:
#Base de datos vectorial
import gdown
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
from sentence_transformers import SentenceTransformer
import numpy as np
import pdfplumber
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import json
#metadatos vectorial

from huggingface_hub import InferenceClient
from langdetect import detect
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from rank_bm25 import BM25Okapi
import re
from nltk.tokenize import word_tokenize
import nltk
from unidecode import unidecode
from llama_index.core.schema import Document


#Base de datos de graphos
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
from rdflib import Graph, Namespace, Literal, RDF, XSD

#base de datos tabulares
import pandas as pd


#base de datos vectorial
from FlagEmbedding import FlagReranker
import pytextrank
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")


#Clasificacion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# LLM
from deep_translator import GoogleTranslator
from google.colab import userdata

#Agente
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.agent.react.base import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.react.formatter import ReActChatFormatter
import wikipedia
nltk.download('punkt_tab')
from llama_index.core.settings import LLM

/usr/local/lib/python3.10/dist-packages


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


### Modelos

In [3]:
model_multi = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

### Api_key

In [4]:
api_key_hf = userdata.get('HUGGINGFACE_TOKEN')

## Extraccion de archivos y web scraping

Las distintas celdas de código están documentadas porque los archivos que se extrajeron usando web scraping, se guardan en una carpeta de Google Drive, para evitar problemas.

### Extraccion de data de documentos

Reglas de manual y opnion desde la pagina web recomendada por la catedra

In [5]:
# url = 'https://misutmeeple.com/2021/04/resena-las-ruinas-perdidas-de-arnak/'
# response = requests.get(url)
# soup = BeautifulSoup(response.content, "html.parser")

# article = soup.find("article", class_="entry content-bg single-entry post-182530 post type-post status-publish format-standard has-post-thumbnail hentry category-resenas tag-colocacion-de-trabajadores tag-construccion-de-mazos tag-draft tag-frantisek-sedlacek tag-gestion-de-la-mano tag-jakub-politzer tag-jiri-kus tag-juegos-de-cartas tag-juegos-de-tablero tag-michaela-min-stachova tag-michal-elwen-stach tag-milan-vavron tag-movimiento-punto-a-punto tag-ondrej-hrdina tag-resenas")
# div_contenidos = article.find("div", class_="entry-content-wrap")
# h2_tags = div_contenidos.find_all("h2")


# rows = []


# for h2 in h2_tags:
#     h2_title = h2.text.strip()  # Título h2
#     paragraphs = []  # Lista para juntar las descripciones
#     h3_descriptions = {}  # Diccionario para los subtítulos h3 y sus descripciones
#     h3_title = None  # Inicializar h3_title antes del ciclo

#     # Buscar los elementos <p>, <h3>, <ol>, <ul> y <h2> siguientes a este h2
#     for sibling in h2.find_next_siblings():
#         if sibling.name == "p":  # Si es un párrafo, lo agregamos a la lista
#             paragraphs.append(sibling.text.strip())
#         elif sibling.name == "h3":  # Si encontramos un h3, almacenamos las descripciones y las vaciamos
#             if paragraphs:
#                 # Juntar todas las descripciones con salto de línea entre ellas
#                 joined_paragraphs = "\n".join(paragraphs)  # Usamos "\n" para separar las descripciones
#                 h3_descriptions[h3_title] = joined_paragraphs  # Guardar como valor para el subtítulo h3
#             h3_title = sibling.text.strip()  # Título del h3
#             paragraphs = []  # Reiniciar la lista de párrafos para el siguiente subtítulo
#         elif sibling.name in ["ol", "ul"]:  # Si encontramos un <ol> o <ul>, capturamos los <li>
#             list_items = []
#             counter = 1  # Inicializamos el contador para los <li>
#             for li in sibling.find_all("li"):
#                 list_items.append(f"# {li.text.strip()}")  # Usamos "#" para cada <li>
#                 counter += 1  # Incrementar el contador
#             # Unir los items de la lista con saltos de línea
#             list_items_text = "\n".join(list_items)
#             paragraphs.append(f"Lista:\n{list_items_text}")  # Agregar la lista a los párrafos
#         elif sibling.name == "h2":  # Si encontramos otro h2, dejamos de capturar párrafos
#             break

#     # Si quedan párrafos después del último h3, agregarlos al diccionario
#     if paragraphs and h3_title:  # Asegurarse de que h3_title esté definido
#         joined_paragraphs = "\n".join(paragraphs)  # Juntar los párrafos restantes con salto de línea
#         h3_descriptions[h3_title] = joined_paragraphs  # Guardar el último conjunto de párrafos bajo h3

#     # Agregar los datos al diccionario de filas
#     for h3, description in h3_descriptions.items():
#         rows.append({"Título": h2_title, "Subtítulo": h3, "Descripción": description})

# # Convertir a DataFrame
# df_contenido = pd.DataFrame(rows)


# url = 'https://misutmeeple.com/2021/04/resena-las-ruinas-perdidas-de-arnak/'

# response = requests.get(url)
# soup = BeautifulSoup(response.content, "html.parser")

# article = soup.find("article", class_="entry content-bg single-entry post-182530 post type-post status-publish format-standard has-post-thumbnail hentry category-resenas tag-colocacion-de-trabajadores tag-construccion-de-mazos tag-draft tag-frantisek-sedlacek tag-gestion-de-la-mano tag-jakub-politzer tag-jiri-kus tag-juegos-de-cartas tag-juegos-de-tablero tag-michaela-min-stachova tag-michal-elwen-stach tag-milan-vavron tag-movimiento-punto-a-punto tag-ondrej-hrdina tag-resenas")
# div_contenidos = article.find("div", class_="entry-content-wrap")
# h2_opinion = div_contenidos.find("h2", string="Opinión Personal")

# h2_opinion = soup.find("h2", string="Opinión Personal")

# opinion_content = ""

# if h2_opinion:
#     for sibling in h2_opinion.find_next_siblings():
#         if sibling.name == "p":
#             opinion_content += sibling.text.strip() + "\n\n"
#         elif sibling.name == "h2":
#             break

# df_opinion = pd.DataFrame([{"Título": "Opinión Personal","Subtítulo": None, "Descripción": opinion_content.strip()}])

# df_concatenado = pd.concat([df_contenido, df_opinion], ignore_index=True)

# with open("output.txt", "w", encoding="utf-8") as file:
#     for index, row in df_concatenado.iterrows():
#         # Escribir el título
#         file.write(f"Título: {row['Título']}\n")

#         # Escribir el subtítulo (puede ser None, en cuyo caso no escribir nada)
#         if row['Subtítulo'] is not None:
#             file.write(f"Subtítulo: {row['Subtítulo']}\n")

#         # Escribir la descripción
#         file.write(f"Descripción: {row['Descripción']}\n")

#         # Añadir una línea en blanco entre cada entrada
#         file.write("\n")


# files.download("output.txt")

Primero se extrae el texto desde la pagina, y luego en el bloque de abajo, se procesa y limpia la informaicon, dejandola en un formato mas comodo para su posterior utilizacion

In [6]:
# # Reglas del manual

# pdf_url = "https://github.com/Nicolas-Noir/NLP-TUIA-2024/blob/main/TP2/Archivos/datos_webscraping/arnak_rules.txt"

# # Descargar el archivo
# response = requests.get(pdf_url)
# pdf_path = "arnak_rules.pdf"
# with open(pdf_path, "wb") as f:
#     f.write(response.content)

# # Extraer texto del PDF
# with pdfplumber.open(pdf_path) as pdf:
#     text = ""
#     for page in pdf.pages:
#         text += page.extract_text()

# with open("arnak_rules.txt", "w", encoding="utf-8") as f:
#     f.write(text)

# lines = [line.strip() for line in text.split("\n") if line.strip()]
# clean_text = "\n".join(lines)

# sections = re.split(r"Capítulo \w+", clean_text)

# lines = text.split("\n")
# paragraphs = []

# for line in lines:
#     if line.strip():  # Solo conservar líneas con texto
#         paragraphs.append(line.strip())

# # Unir las líneas en párrafos
# text_cleaned = " ".join(paragraphs)

# with open("arnak_rules.txt", "w", encoding="utf-8") as file:
#     file.write(text_cleaned)

# files.download("arnak_rules.txt")

Paginas sobre opiniones en ingles

In [7]:
# url = "https://www.boardgamequest.com/the-lost-ruins-of-arnak-review/"

# response = requests.get(url)
# soup = BeautifulSoup(response.content, "html.parser")

# headers = soup.find_all(["h3", "p"])

# content_list = []

# for element in headers:
#     if 'You seen most things before' in element.text.strip():
#         break  # Stop if the phrase is found
#     if element.name == "h3":
#         # Add a newline before the header
#         content_list.append("\n" + element.text.strip())
#     elif element.name == "p":
#         # Add paragraph text
#         content_list.append(element.text.strip())

# # Join the content list into a single string
# result = "\n".join(content_list)

# with open("arnak_opinion.txt", "w", encoding="utf-8") as file:
#     file.write(result)

Otra pagina de opinion y contenidos varios del juego

In [8]:
# driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver-linux64/chromedriver"), options=options)

# url = "https://tabletopbellhop.com/game-reviews/lost-ruins-of-arnak/"
# driver.get(url)

# # Find all <p> elements
# paragraphs = driver.find_elements(By.TAG_NAME, "p")

# # Extract the text from each <p> element
# paragraph_texts = []
# for p in paragraphs:
#     if p.text.strip() == 'What’s the last game you played that wowed you or totally blew away your expectations? Tell us about it in the comments below!':
#         break
#     paragraph_texts.append(p.text.strip())

# driver.quit()

# with open("arnak_opinion.txt", "w", encoding="utf-8") as file:
#     file.write("\n".join(paragraph_texts))

Pagina de opinion sencilla con expericinecas sobre el juego

In [9]:
# import requests
# from bs4 import BeautifulSoup

# url = "https://www.boardgamequest.com/the-lost-ruins-of-arnak-review/"

# response = requests.get(url)

# soup = BeautifulSoup(response.content, "html.parser")

# paragraphs = soup.find_all("p")
# paragraph_texts = [p.text.strip() for p in paragraphs if p.text.strip()]

# headers = soup.find_all("h3")
# header_texts = [h3.text.strip() for h3 in headers if h3.text.strip()]

# with open("arnak_opinion.txt", "w", encoding="utf-8") as file:
#     file.write("\n".join(paragraph_texts))



El resto de archivos guardados en googledrive son pdfs encontrados en la pagina principal del juego

### Extraccion de data de grafos

In [10]:
# options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")

# driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver-linux64/chromedriver"), options=options)

# driver.get('https://boardgamegeek.com/boardgame/312484/lost-ruins-of-arnak/credits')

# data = {}

# # Encuentra todos los elementos <li> en la lista principal
# list_items = driver.find_elements(By.CSS_SELECTOR, "ul.outline-col-xs-block li")

# # Iterar sobre cada categoría (elementos <li>)
# for item in list_items:
#     # Extraer el título de la categoría (si existe)
#     category_title = item.find_element(By.CSS_SELECTOR, ".outline-item-title span.ng-binding").text.strip()

#     # Extraer el contenedor de los valores
#     values_container = item.find_element(By.CSS_SELECTOR, ".outline-item-description")

#     # Capturar todos los elementos hijos y sus textos
#     values = values_container.find_elements(By.CSS_SELECTOR, "*")

#     # Si la categoría no existe en el diccionario, inicializarla
#     if category_title not in data:
#         data[category_title] = []

#     # Procesar los valores
#     for value in values:
#         text = value.text.strip()

#         # Dividir cadenas que contengan saltos de línea en elementos individuales
#         split_values = text.split('\n')

#         # Agregar cada elemento único a la lista de la categoría
#         for val in split_values:
#             val = val.strip()
#             if val and val not in data[category_title]:  # Verificar duplicados
#                 data[category_title].append(val)


# driver.get('https://de.wikipedia.org/wiki/Die_verlorenen_Ruinen_von_Arnak')

# list_items = driver.find_elements(By.CSS_SELECTOR, "ul li")

# premios = []

# # Iterar sobre cada <li> encontrado
# for item in list_items:
#     # Extraer el texto visible (nombre)
#     if len(premios) <= 5:
#       name = item.text.strip()
#       premios.append(name)

# data['Awards'] = premios

# with open('data.json', 'w', encoding='utf-8') as file:
#     json.dump(data, file, ensure_ascii=False, indent=4)

Extraccion de datos tabulares

En este caso son 2 archivos en formato XLSM que se encontraorn en la misma pagina web de los juegos que los documentos de texto

### Creacion de DataFrame con preguntas para entrenar el modelo de clasificacion

In [11]:
# texto = [
#     # Strategies
#     "What strategies are most effective for winning?",
#     "What is the best strategy if I'm playing with inexperience?",
#     "What strategies work best if I already have some experience?",
#     "How can I optimize my strategy as an experienced player?",
#     "What are the best strategies for beginners?",
#     "What strategies are recommended for advanced games?",
#     "How does the use of specific cards affect the overall strategy?",
#     "How can I balance exploration and resource gathering?",
#     "What is the best way to gain victory points quickly?",
#     "How can I counteract other players' strategies?",
#     "What elements of the board are key to a winning strategy?",
#     "How can I make the most of idols in the game?",
#     "What is the most efficient way to manage resources during the game?",
#     "How can I adapt my strategy to unpredictable board changes?",
#     "What strategies are recommended for competitive gameplay?",
#     "How can I plan long-term strategies in games with many players?",
#     "What is the best approach for minimizing risks in gameplay?",
#     "What tactics work best in the early stages of the game?",
#     "How can I maximize my score by focusing on secondary objectives?",
#     "What strategy should I use if I’m falling behind in points?",
#     "How can I effectively counter aggressive opponents?",
#     "What balance should I aim for between offense and defense?",
#     # Game Rules
#     "What are the basic rules of the game?",
#     "What should I do if I don't understand a specific rule?",
#     "Are there variations in the rules depending on the number of players?",
#     "How many players can play the game?",
#     "How do I set up games depending on the number of players?",
#     "What are the rules for each player ... do if two players tie in points?",
#     "How do the exploration and excavation actions work?",
#     "How many actions can a player take on their turn?",
#     "What are the conditions for winning the game?",
#     "What should I do if I lose all my cards in a round?",
#     "How are special rules handled in solo mode?",
#     "What happens if there are not enough resources available for all players?",
#     "What happens if a player runs out of resources mid-turn?",
#     "What are the rules for ties at the end of the game?",
#     "How do bonuses from special tiles work?",
#     "What are the specific rules for using advanced player abilities?",
#     "How are penalties applied for breaking game rules?",
#     "What is the sequence of actions during a player’s turn?",
#     "What are the setup differences for different player counts?",
#     "What are the specific rules for scoring at the end of the game?",
#     "How do temporary effects from cards or abilities work?",
#     "What happens if the resource pool is depleted during gameplay?",
#     # Turn dynamics
#     "What can I do on each turn to maximize my performance?",
#     "How should I manage my resources during a turn?",
#     "What actions are available to a player on each turn?",
#     "How can I best manage my turn to take advantage of resources?",
#     "When is the best time to use a research action?",
#     "How can I maximize my points in the later turns of the game?",
#     "What role does the use of artifacts play in the turn dynamics?",
#     "How long does an average turn last in games with more than four players?",
#     "What is the best way to prepare for your next turn?",
#     "How can I optimize my turn for resource generation?",
#     "What actions provide the most value in early turns?",
#     "What role do timing and order of actions play in each turn?",
#     "How can I plan my actions to disrupt opponents’ strategies?",
#     "What is the importance of planning multiple turns ahead?",
#     "How can I effectively use special actions during my turn?",
#     "When should I prioritize collecting resources over taking risks?",
#     "What are the best tactics for managing time-limited turns?",
#     "How do turn dynamics differ with fewer players?",
#     # Cards
#     "What do different cards do in the game?",
#     "What card is the most powerful in a specific situation?",
#     "How can I use cards defensively?",
#     "What are the most valuable cards to gain strategic advantages?",
#     "How can I optimize the use of cards in combination with artifacts?",
#     "What types of cards are in the game and how are they used?",
#     "What effects do fear cards have and how are they handled?",
#     "How do purchased cards affect the development of the game?",
#     "When is it better to invest in cards instead of physical resources?",
#     "Which cards are most useful for advancing the research track?",
#     "What are the best card combinations for maximizing synergy?",
#     "How can I recover from drawing disadvantageous cards?",
#     "What is the optimal way to cycle through my deck?",
#     "How do cards with dual effects influence gameplay strategy?",
#     "What cards should be prioritized in the late game?",
#     "How do cards affect the flow of resources in the game?",
#     "What are the benefits of drafting cards early in the game?",
#     "How can I best utilize cards with high-cost effects?",
#     "When should I save powerful cards for later rounds?",
#     "What is the impact of fear cards on a player's strategy?",
#     # Advantages
#     "What are the advantages of using the Pickaxe card?",
#     "What are the advantages of using the Bow and Arrows card?",
#     "What are the advantages of using the Dog card?",
#     "What are the advantages of using the Fishing Rod card?",
#     "What are the advantages of using the Chronometer card?",
#     "How to use the Hat card?",
#     "How to use the Trowel card?",
#     "How to use the Gold Pan card?",
#     "When to use the Brush card?",
#     "when to use the Brush card?",
#     "when to use the Dog card?",
#     "How can I best leverage the Spyglass card for exploration?",
#     "What are the advantages of using multi-purpose cards?",
#     "How do artifacts enhance my chances of winning?",
#     "What are the advantages of accumulating rare resources?",
#     "How can I utilize the advantages of cooperative cards?",
#     "What are the benefits of focusing on resource-efficient strategies?",
#     "How do area-specific bonuses affect gameplay advantages?",
#     "What are the advantages of early exploration actions?",
#     "What benefits come from prioritizing scoring cards?",
#     "How can the right equipment amplify a player’s strengths?",
#     # Disadvantages
#     "What are the disadvantages of the Pidgeon card?",
#     "What are the disadvantages of the Torch card?",
#     "What are the disadvantages of the Axe card?",
#     "What are the disadvantages of the Parrot card?",
#     "What are the disadvantages of the Watch card?",
#     "how to use the Horse card?",
#     "how to use the Precision Compass card?",
#     "how to use the Gold Pan card?",
#     "when to use the Tent card?",
#     "when to use the Rope card?",
#     "when to use the Automobile card?",
#     "What are the risks of holding too many fear cards?",
#     "How can a poorly balanced strategy disadvantage a player?",
#     "What disadvantages come with focusing too heavily on one mechanic?",
#     "How does an overly defensive strategy hinder progress?",
#     "What challenges arise from overcommitting to a single resource type?",
#     "What are the drawbacks of rushing to collect victory points?",
#     "How do limited actions per turn create disadvantages?",
#     "What penalties exist for poor resource management?",
#     "What are the downsides of ignoring exploration mechanics?",
#     "How can relying on luck-based cards backfire?",
#     # Why is it useful
#     "Why is the Binoculars card useful?",
#     "Why is the Rough Map card useful?",
#     "Why is the Pack Donkey card useful?",
#     "Why is the Ostrich card useful?",
#     "Why is the Sea Turtle card useful?",
#     "How to use the Lantern card?",
#     "How to use the Large Backpack card?",
#     "How to use the Army Knife card?",
#     "When to use the Revolver card?",
#     "When to use the Grappling Hook card?",
#     "When to use the Airdrop card?",
#     "Why is the Pickaxe card essential for resource gathering?",
#     "Why is the Compass card valuable for planning exploration?",
#     "How does the Torch card contribute to uncovering hidden resources?",
#     "Why is the Research Journal card effective for advancing tracks?",
#     "What makes the Artifact Collection card a game-changer?",
#     "How does the Expansion Pack card enhance gameplay options?",
#     "Why is the Resource Exchange card crucial in tight situations?",
#     "How can the Relic Finder card optimize exploration?",
#     "Why is the Victory Multiplier card beneficial in late-game scoring?",
#     "What makes the Special Ability card a strategic asset?",
#     # Variation
#     "What is the story behind the game 'Lost Ruins of Arnak'?",
#     "What theme does the game have and how is it reflected in the mechanics?",
#     "What inspires the visual and narrative design of the game?",
#     "What are the key elements of adventure in the game?",
#     "What similarities does this game have with other games of its genre?",
#     "How does exploration relate to scoring points?",
#     "What challenges do the ruins present to players?",
#     "What are the differences between single player and multiplayer mode?",
#     "What are the variations in difficulty level?",
#     "What rules change in two-player games?",
#     "How is the game balance adjusted for games with more than four players?",
#     "What expansions are there for this game and how do they change the mechanics?",
#     "How does the solo mode change the dynamics of the game?",
#     "What are the differences between the base game and expansions?",
#     "What unique challenges does the advanced difficulty present?",
#     "How do the mechanics adapt to games with different player counts?",
#     "What are the thematic inspirations behind the game design?",
#     "How do the new expansions alter resource management mechanics?",
#     "What adjustments are made for balancing competitive multiplayer?",
#     "What are the major differences between cooperative and solo modes?",
#     "How do the rules change in tournament play?",
#     "What influences from other genres are present in this game?"
# ]

# grafo = [
#     "What is the relationship between the game designers?",
#     "What awards did the game designers win?",
#     "What connection do the designers have with the artists involved?",
#     "Who are the artists involved in this game?",
#     "Who are the designers involved in this game?",
#     "Who are the graphic designers involved in this game?",
#     "What are the different names of the game in other languages?",
#     "What categories does this game have according to the official classification?",
#     "What game family does it belong to?",
#     "What mechanics does the game have?",
#     "What awards did the game win",
#     "What awards did the game developers win",
#     "Who are the official distributors of the game?",
#     "What year was the game released?",
#     "What are the names of the official distributors of the game?",
#     "What are the categories that the game has?",
#     "What are the mechanics of the game?",
#     "What are the names of the game designers?",
#     "What are the names of the game's graphic designers?",
#     "What are the game mechanics?",
#     "What are the names of the families associated with the game?",
#     "What are the game mechanics?",
#     "What are the names of the game's dealers?",
#     "What are the names of the awards that the game designers won?",
#     "What are the names of the prizes the game won?",
#     "when was created the game?",
#     "What awards has the game won?",
#     "Who are the artists credited for this game?",
#     "Who are the developers behind the game mechanics?",
#     "What is the relationship between the game designers and the artists?",
#     "What is the origin of the game's theme?",
#     "What awards have the graphic designers of the game received?",
#     "Who are the main collaborators for this game?",
#     "What family of games does this belong to?",
#     "What languages has the game been translated into?",
#     "What categories does the game belong to?",
#     "What year was the game officially published?",
#     "Who are the publishers involved in the game?",
#     "What are the mechanics classified under this game?",
#     "What is the official classification of the game's genre?",
#     "Who are the main distributors of the game?",
#     "Who worked on the graphic design for the game?",
#     "What is the artistic style of the game?",
#     "What inspired the visual elements of the game?",
#     "What role do the artists play in shaping the game's identity?",
#     "What unique mechanics differentiate this game from others?",
#     "What themes does the game explore?",
#     "What connections exist between the game designers and publishers?",
#     "What prizes have the game's designers achieved?",
#     "What awards has the game itself achieved?",
#     "What are the official names of the game in different regions?",
#     "What subjects are covered in the game’s mechanics?",
#     "What family of games does this title fall under?",
#     "What awards have been received by the game's developers?",
#     "What is the connection between the designers and the distributors?",
#     "What design choices make this game unique?",
#     "What are the mechanics defined by the creators?",
#     "What is the background of the game designers?",
#     "Who are the designers most associated with this game?",
#     "What roles do the graphic designers play in the production?",
#     "Who distributed the game internationally?",
#     "What are the local names of the game in different countries?",
#     "What mechanics make this game popular among players?",
#     "What prizes have been won by the artists of the game?",
#     "What are the names of the artists who illustrated the game?",
#     "What connections do the artists have with the game designers?",
#     "What languages are available for this game?",
#     "What themes or subjects are covered in the game design?",
#     "What official recognitions has the game received?",
#     "Who are the lead contributors to the game’s design?",
#     "What are the primary features of the game mechanics?",
#     "What categories is the game officially listed under?",
#     "What are the names of the designers and developers?",
#     "What are the names of the game's distributors?",
#     "What families of games include this title?",
#     "What is the game’s relationship to others in its category?",
#     "What are the translated names of this game?",
#     "What artistic elements are emphasized in the game?",
#     "What key mechanics are highlighted in reviews of the game?",
#     "What awards have recognized the artistic elements of the game?",
#     "What are the mechanics most praised by the players?",
#     "What categories does the game belong to according to critics?",
#     "What relationships exist between the game’s developers and publishers?",
#     "What artistic influences shaped the game’s visuals?",
#     "What are the game mechanics introduced in this version?",
#     "What is the relationship between the game designers and publishers?",
#     "What awards has the game won over the years?",
#     "Who are the main artists involved in the creation of the game?",
#     "What are the notable categories assigned to the game?",
#     "What are the different languages the game has been translated into?",
#     "What family of games does this title belong to?",
#     "Who are the designers credited for the game mechanics?",
#     "What collaborations were involved in creating the game?",
#     "Who are the main distributors of the game globally?",
#     "What prizes did the artists win for their work on the game?",
#     "What inspired the mechanics used in the game?",
#     "What awards have the game’s designers received individually?",
#     "What themes does the game explore through its design?",
#     "What are the official mechanics classified under this game?",
#     "What unique features distinguish the game’s categories?",
#     "Who illustrated the game's visual elements?",
#     "What role did the graphic designers play in the project?",
#     "What are the mechanics praised by professional reviewers?",
#     "Who are the distributors responsible for the game’s international success?",
#     "What mechanics are specifically designed for competitive play?",
#     "What families of games share similar mechanics to this one?",
#     "What artistic elements make this game stand out?",
#     "Who are the key artists responsible for the game’s visual style?",
#     "What are the translated names of the game in European countries?",
#     "What relationship do the game developers have with the graphic designers?",
#     "What mechanics define the game’s unique play experience?",
#     "What categories is the game listed under on major platforms?",
#     "What are the themes highlighted in the game’s rulebook?",
#     "Who were the contributors to the game’s graphic design?",
#     "What partnerships existed between the designers and distributors?",
#     "What are the names of the official game distributors?",
#     "What artistic achievements have been recognized in the game?",
#     "What are the most distinctive aspects of the game mechanics?",
#     "Who are the graphic designers that contributed to the game?",
#     "What awards were given to the game's publishers?",
#     "What categories is the game classified under by players?",
#     "What are the main mechanics used in the advanced version of the game?",
#     "Who are the main graphic contributors for the special edition?",
#     "What are the regional differences in the game’s design?",
#     "What key elements differentiate the game’s mechanics from others?",
#     "What are the names of the key contributors to the game mechanics?",
#     "Who are the artists responsible for the cover art?",
#     "What themes or categories were influenced by the game designers?",
#     "What languages have localized editions of the game?",
#     "What families of games influenced this title’s creation?",
#     "What are the unique mechanics introduced in the expansion packs?",
#     "What relationships exist between the game’s designers and artists?",
#     "What themes are central to the game’s storytelling?",
#     "What awards recognize the game's mechanics?",
#     "Who were the key players in the game’s publishing process?",
#     "What mechanics are designed for cooperative gameplay?",
#     "What families of games does this one derive inspiration from?",
#     "What are the names of the official expansion packs for the game?",
#     "What artistic techniques are used in the game’s design?",
#     "What are the primary mechanics highlighted in critical reviews?",
#     "Who are the graphic designers responsible for the updated version?",
#     "What partnerships contributed to the game’s production?",
#     "What categories does the game belong to according to its creators?",
#     "What are the relationships between the game’s various contributors?",
#     "What awards recognize the game’s artistic achievements?",
#     "What languages were prioritized for the game’s localization?"

# ]

# tabular = [
#     # VP Efficiency
#     "What is the VP efficiency of the Pickaxe card?",
#     "What is the VP efficiency of the Bow and Arrows card?",
#     "What is the VP efficiency of the Trowel card?",
#     "What is the VP efficiency of the Gold Pan card?",
#     "What is the VP efficiency of the Sturdy Boots card?",
#     "What is the VP efficiency of the Hat card?",
#     "What is the VP efficiency of the Fishing Rod card?",
#     "What is the VP efficiency of the Chronometer card?",
#     "What is the VP efficiency of the Compass card?",
#     "What is the VP efficiency of the Pack Donkey card?",
#     "What is the VP efficiency of the Ostrich card?",
#     "What is the VP efficiency of the Army Knife card?",
#     "What is the VP efficiency of the Binoculars card?",
#     "What is the VP efficiency of the Lantern card?",
#     "What is the VP efficiency of the Large Backpack card?",
#     "What is the VP efficiency of the Revolver card?",
#     "What is the VP efficiency of the Grappling Hook card?",
#     "What is the VP efficiency of the Airdrop card?",
#     # Efficiency
#     "What is the efficiency of the Trowel card?",
#     "What is the efficiency of the Gold Pan card?",
#     "What is the efficiency of the Sturdy Boots card?",
#     "What is the efficiency of the Brush card?",
#     "What is the efficiency of the Dog card?",
#     "What is the efficiency of the Chronometer card?",
#     "What is the efficiency of the Carrier card?",
#     "What is the efficiency of the Pickaxe?",
#     "What is the efficiency of the Army Knife card?",
#     "What is the efficiency of the Revolver card?",
#     "What is the efficiency of the Grappling Hook card?",
#     "What is the efficiency of the Airdrop card?",
#     "What is the efficiency of the Ostrich card?",
#     "What is the efficiency of the Sea Turtle card?",
#     "What is the efficiency of the Sturdy Boots card?",
#     "What is the efficiency of the Machete card?",
#     "What is the efficiency of the Lantern card?",
#     "What is the efficiency of the Large Backpack card?",
#     # Difference
#     "What is the difference in the Brush card?"
#     "What is the difference in the Dog card?",
#     "What is the difference between the Fishing Rod card?",
#     "What is the difference between the Chronometer card?",
#     "What is the difference between the Aeroplane card?",
#     "What is the difference between the Theodolite card?",
#     "What is the difference between the Carrier card?",
#     "What is the difference between the Bow and Arrows card?",
#     "What is the difference between the Axe card?",
#     "What is the difference between the Compass card and the Theodolite card?",
#     "What is the difference between the Lantern card and the Torch card?",
#     "What is the difference between the Pack Donkey card and the Horse card?",
#     "What is the difference between the Ostrich card and the Sea Turtle card?",
#     "What is the difference between the Grappling Hook card and the Rope card?",
#     "What is the difference between the Revolver card and the Bow and Arrows card?",
#     "What is the difference between the Airdrop card and the Steam Boat card?",
#     "What is the difference between the Tent card and the Rope card?",
#     "What is the difference between the Pidgeon card and the Parrot card?",
#     "What is the difference between the Automobile card and the Boat card?",
#     # Cost
#     "What is the cost of the Aeroplane card?",
#     "What is the cost of the Theodolite card?",
#     "What is the cost of the Carrier card?",
#     "What is the cost of the Pidgeon card?",
#     "What is the cost of the Torch card?",
#     "What is the cost of the Axe card?",
#     "What is the cost of the Bow and Arrows card?",
#     "What is the cost of the Brush card?",
#     "What is the cost of the Large Backpack card?",
#     "What is the cost of the Lantern card?",
#     "What is the cost of the Ostrich card?",
#     "What is the cost of the Army Knife card?",
#     "What is the cost of the Revolver card?",
#     "What is the cost of the Grappling Hook card?",
#     "What is the cost of the Sturdy Boots card?",
#     "What is the cost of the Sea Turtle card?",
#     "What is the cost of the Steam Boat card?",
#     "What is the cost of the Airdrop card?",
#     # Benefit
#     "What benefits does the Pidgeon card provide?",
#     "What benefits does the Torch card in certain strategies?",
#     "What is the benefit of using the Axe card?",
#     "What is the benefit of using the Parrot card?",
#     "What is the benefit of using the Aeroplane card?",
#     "What is the benefit of using the Trowel card?",
#     "What is the benefit of using the Carrier card?",
#     "What is the benefit of using the Pack Donkey card?",
#     "What benefits does the Lantern card provide?",
#     "What benefits does the Large Backpack card offer?",
#     "What is the benefit of using the Army Knife card?",
#     "What is the benefit of using the Binoculars card?",
#     "What is the benefit of using the Compass card?",
#     "What is the benefit of using the Machete card?",
#     "What is the benefit of using the Revolver card?",
#     "What is the benefit of using the Ostrich card?",
#     "What is the benefit of using the Airdrop card?",
#     "What is the benefit of using the Rope card?",
#     # Recursive Inflation
#     "How does recursive inflation affect the Parrot card?",
#     "What is the impact of recursive inflation on the Watch card?",
#     "What is recursive inflation for the Axe card?",
#     "What is recursive inflation for the Dog card?",
#     "What is recursive inflation for the Theodolite card?",
#     "What is recursive inflation for the Chronometer card?",
#     "What is recursive inflation for the Torch card?",
#     "What is recursive inflation for the Horse card?",
#     "How does recursive inflation affect the Large Backpack card?",
#     "What is the impact of recursive inflation on the Grappling Hook card?",
#     "What is recursive inflation for the Airdrop card?",
#     "What is recursive inflation for the Steam Boat card?",
#     "What is recursive inflation for the Sturdy Boots card?",
#     "What is recursive inflation for the Lantern card?",
#     "What is recursive inflation for the Machete card?",
#     "What is recursive inflation for the Army Knife card?",
#     "What is recursive inflation for the Binoculars card?",
#     "What is recursive inflation for the Compass card?",
#     # Notes
#     "The Horse card has notes associated with it?",
#     "What notes does the Precision Compass card have?",
#     "Does the Parrot card have notes associated with it?",
#     "Does the Watch card have notes associated with it?",
#     "Does the Torch card have notes associated with it?",
#     "Does the Axe card have notes associated with it?",
#     "What notes are associated with the Large Backpack card?",
#     "What notes does the Grappling Hook card include?",
#     "Does the Revolver card have any special notes?",
#     "Does the Sea Turtle card come with strategic notes?",
#     "What notes are tied to the Ostrich card?",
#     "Does the Compass card have any associated notes?",
#     "Are there any notes for the Airdrop card?",
#     "What notes are relevant to the Steam Boat card?",
#     "What notes apply to the Binoculars card?",
#     "Does the Lantern card have any specific notes?",
#     # Value of playing it
#     "What is the value of the Tent card?",
#     "What is the value of the Rope card?",
#     "What is the value of the Gold Pan card?",
#     "What is the value of the Compass card?",
#     "What is the value of the Parrot card?",
#     "What is the value of the Pidgeon card?",
#     "What is the value of the Horse card?",
#     # Points
#     "How many points can the Automobile card award?",
#     "How many points does the Steam Boat card offer?",
#     "How many points does the Rope card?",
#     "How many points does the Tent card have?",
#     "How many points does the Trowel card have?",
#     "How many points does the Pack Donkey card have?",
#     "How many points does the Horse card have?",
#     # Speed
#     "What speed does the card have?",
#     "How many points ... Machete?",
#     "What is the speed of the Binoculars card?",
#     "What is the speed of the Automobile card?",
#     "What is the speed of the Boat card?",
#     "What is the speed of the Trowel card?",
#     "What is the speed of the Horse card?",
#     "What is the speed of the Rope card?",
#     # Versatility
#     "What is the versatility of the Rough Map card?",
#     "What is the versatility of the Pack Donkey card?",
#     "What is the versatility of the Machete card?",
#     "What is the versatility of the Horse card?",
#     "What is the versatility of the Rope card?",
#     "What is the versatility of the Pidgeon card?",
#     "What is the versatility of the Chronometer card?",
#     "What is the versatility of the Grappling Hook card?",
#     "What is the versatility of the Rope card?",
#     "What is the versatility of the Lantern card?",
#     "What is the versatility of the Large Backpack card?",
#     "What is the versatility of the Army Knife card?",
#     "What is the versatility of the Ostrich card?",
#     "What is the versatility of the Sea Turtle card?",
#     "What is the versatility of the Sturdy Boots card?",
#     "What is the versatility of the Machete card?",
#     "What is the versatility of the Compass card?",
#     # Best Use
#     "What is the best use of the Ostrich card?",
#     "How can you best use the Sea Turtle card?",
#     "What is the best use of the Pickaxe card?",
#     "What is the best use of the Automobile card?",
#     "What is the best use of the Axe?",
#     "What is the best use of the AParrotxe card?",
#     "What is the best use of the Chronometer card?",
# ]


# dataset_entrenamiento = []

# for pregunta in texto:
#   dataset_entrenamiento.append((0,pregunta))

# for pregunta in grafo:
#   dataset_entrenamiento.append((1,pregunta))

# for pregunta in tabular:
#   dataset_entrenamiento.append((2,pregunta))

# querys_entrenamiento = pd.DataFrame(dataset_entrenamiento, columns=['clase', 'pregunta'])

# from google.colab import files

# querys_entrenamiento.to_csv('querys_entrenamiento.csv', index=False)

# files.download('querys_entrenamiento.csv')

## Carga de pdf y archivos csv

In [12]:
# Se cargan los archivos PDF y CSV desde la carpeta en Google Drive
url = 'https://drive.google.com/drive/folders/1K5FxxNzHG86gb2b6xRjuQS74wVL2woab?usp=sharing'

gdown.download_folder(url, quiet=True, output='lost-ruins-of-arnak')

['lost-ruins-of-arnak/A Plain and Simple Guide.pdf',
 'lost-ruins-of-arnak/Best first movment.pdf',
 'lost-ruins-of-arnak/Complete strategy guide Beginner-Average-Experienced-Expert.pdf',
 'lost-ruins-of-arnak/Content text and opinion.pdf',
 'lost-ruins-of-arnak/csv Lost Ruins of Arnak - Master Sheet - Items.csv',
 'lost-ruins-of-arnak/csv tools_dataset.csv',
 'lost-ruins-of-arnak/Different reviews and opnions.pdf',
 'lost-ruins-of-arnak/guide to Lost Ruins of Arnak for experts_chunks.pdf',
 'lost-ruins-of-arnak/Rules_summary.pdf',
 'lost-ruins-of-arnak/Spotlight of some cards.pdf']

## Base de datos vectorial

No es necesario ejecutar esta celda, ya que los datos de los chunks y embeddings están guardados en GitHub, para que no sea necesario esperar el proceso de creación de los mismos.

In [13]:
# # Modelo de embeddings
# model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
# nlp = spacy.load("en_core_web_sm")

# # Extraccion del textod e los PDFs
# def extrae_texto_plumber(pdf):
#   """
#   Extrae el texto de un archivo PDF utilizando la biblioteca pdfplumber.
#   """
#   with pdfplumber.open(pdf) as pdf_file:
#     texto = ''.join([page.extract_text() for page in pdf_file.pages])
#   return texto

# # Limpieza del texto
# def clean_text_with_newlines(text):
#   """
#   Limpia el texto eliminando líneas vacías y espacios innecesarios al principio y al final de cada línea.
#   """
#   lines = text.splitlines()
#   cleaned_text = ' '.join(line.strip() for line in lines if line.strip())
#   return cleaned_text

# def remove_extra_spaces(text):
#   """
#   Elimina espacios adicionales entre palabras en un texto.
#   """
#   return ' '.join(text.split())


# def agrupar_por_similitud_con_ventanas(texto, max_caracteres_chunk=1300, min_caracteres_chunk=1000, ventana_size=3, umbral_similitud=0.7):
#   """
#   Crea chunks y embeddings de los archivos PDF utilizando similitud de coseno.
#   """
#   # Se divide el texto en oraciones utilizando spaCy
#   doc = nlp(texto)
#   oraciones = [sent.text.strip() for sent in doc.sents]
#   print(f"\nTotal de oraciones: {len(oraciones)}")
#   print("Ejemplo de oraciones:", oraciones[:5], "...\n")

#   # Calcular embeddings para cada oración
#   embeddings_oraciones = model_multi.encode(oraciones)
#   print(f"Shape de embeddings de oraciones: {embeddings_oraciones.shape}")

#   # Calcular la matriz de similitud
#   matriz_similitud = cosine_similarity(embeddings_oraciones)
#   print("\nMatriz de similitud:")
#   for i in range(min(5, len(oraciones))):
#     print(f"Oración {i}: {matriz_similitud[i, :5]} ...")

#   # Inicializar variables
#   chunks = []
#   chunk_actual = []
#   caracteres_actuales = 0

#   # Procesar las oraciones
#   for i in range(len(oraciones)):
#     print(f"\nProcesando oración {i}: {oraciones[i]}")
#     chunk_actual.append(oraciones[i])
#     caracteres_actuales += len(oraciones[i])
#     print(f"Caracteres acumulados en chunk actual: {caracteres_actuales}")

#     # Calcular similitud promedio en la ventana deslizante
#     indices_ventana = range(max(0, i - ventana_size), min(len(oraciones), i + ventana_size + 1))
#     print(f"Índices de la ventana deslizante: {list(indices_ventana)}")
#     similitudes = [matriz_similitud[i, j] for j in indices_ventana if i != j]
#     print(f"Similitudes en la ventana: {similitudes}")

#     if similitudes:
#       promedio_similitud = np.mean(similitudes)
#     else:
#       promedio_similitud = 0

#     print(f"Promedio de similitud en la ventana: {promedio_similitud}")

#         # Condición para cerrar el chunk
#     if (caracteres_actuales >= max_caracteres_chunk or (promedio_similitud < umbral_similitud and caracteres_actuales >= min_caracteres_chunk)):
#       print("Condición cumplida para cerrar el chunk.")
#       chunks.append(" ".join(chunk_actual))
#       print(f"Chunk cerrado con {len(chunk_actual)} oraciones.")
#       chunk_actual, caracteres_actuales = [], 0

#     # Agregar último chunk si queda algo pendiente
#   if chunk_actual and caracteres_actuales >= min_caracteres_chunk:
#     chunks.append(" ".join(chunk_actual))

#     print(f"\nÚltimo chunk agregado con {len(chunk_actual)} oraciones.")

#   # Calcular embeddings para los chunks
#   embeddings_chunks = model_multi.encode(chunks)

#   print(f"\nTotal de chunks creados: {len(chunks)}")
#   print("Ejemplo de chunks:", chunks[:3], "...\n")

#   return chunks, embeddings_chunks


# # Se ordenan los archivos por nombre
# carpeta_pdfs = 'lost-ruins-of-arnak'
# archivos_pdfs = sorted(os.listdir(carpeta_pdfs))
# lista_embeding_chunks = []

# # Se itera sobre cada archivo creando sus chunks y embeddings
# for archivo_pdf in archivos_pdfs:
#     chunks_y_embeddings = []
#     if archivo_pdf.endswith('.pdf'):
#         print(f"Procesando: {archivo_pdf}")

#         # Construir la ruta completa del archivo PDF
#         ruta_pdf = os.path.join(carpeta_pdfs, archivo_pdf)

#         # Extraer el texto del PDF
#         texto = extrae_texto_plumber(ruta_pdf)

#         # Limpiar el texto (eliminar saltos de línea y espacios extra)
#         texto_limpio = clean_text_with_newlines(texto)
#         texto_limpio = remove_extra_spaces(texto_limpio)

#         # Agrupar las oraciones semánticamente y obtener los chunks y embeddings
#         chunks, embeddings = agrupar_por_similitud_con_ventanas(texto_limpio, max_caracteres_chunk=1300, min_caracteres_chunk=1000, ventana_size=3, umbral_similitud=0.7)

#         # Almacenar los chunks y embeddings en la lista con el nombre del archivo
#         for chunk, embedding in zip(chunks, embeddings):
#             chunks_y_embeddings.append((archivo_pdf, chunk, embedding))

#         lista_embeding_chunks.append(chunks_y_embeddings)

# # Guardar los datos en un archivo pickle para no tener que ejecutar el proceso cada vez que se inicie el programa
# output_file = "lista_embedding_chunks.pkl"

# with open(output_file, "wb") as f:
#     pickle.dump(lista_embeding_chunks, f)

### Cargar los chunks y embeddings

In [14]:
url_vectorial = "https://github.com/Nicolas-Noir/NLP-TUIA-2024/raw/main/TP2/Archivos/datos_tabulares/lista_embedding_chunks.pkl"

response = requests.get(url_vectorial)

with open("lista_embedding_chunks.pkl", "wb") as f:
    f.write(response.content)

with open("lista_embedding_chunks.pkl", "rb") as f:
    lista_embedding_chunks = pickle.load(f)

### Creacion y carga de datos en ChromaDB

In [15]:
# Primero se inicializa la base de datos Chromadb de forma persistente, y luego se le agregan los chunks, embeddings y un, id identificatorio por PDF y chunk
client = chromadb.PersistentClient(path="./bdd_chormadb")

collection = client.get_or_create_collection(name="bdd_archivos")

for doc_id, (chunk_embedding, metadata) in enumerate(zip(lista_embedding_chunks, _), start=1):
    archivo, chunks, embeddings = zip(*chunk_embedding)
    num_chunks = len(chunks)

    collection.add(
        documents=list(chunks),
        embeddings=list(embeddings),
        ids=[f"doc{doc_id}_chunk{i}" for i in range(num_chunks)])

## Base de datos de graphos

In [16]:
url_grafos = "https://github.com/Nicolas-Noir/NLP-TUIA-2024/raw/main/TP2/Archivos/datos_grafos/data.json"

grafo_json = requests.get(url_grafos)

with open('data.json', 'w', encoding='utf-8') as file:
        file.write(grafo_json.text)

with open('data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [17]:
def ex_url(categoria,nombre):
  """
  Se modifican los nombres para pasarle un formato de URL a los nodos de los grafos.
  """
  EX = Namespace("http://arnak.local/")
  nombre = nombre.replace(" ", "_")
  return EX[categoria + '/' + nombre]

# Se inicializa un objeto de grafo
g = Graph()

# Namespace local
EX = Namespace("http://arnak.local/")

g.bind("ex", EX)

game_uri = EX["game/Lost_Ruins_of_Arnak"]

designers = {}
for design in data['Designers']:
   designers[design] = ex_url('designer',design)

artists = {}
for artist in data['Artists']:
   artists[artist] = ex_url('artist',artist)

publishers = {}
for publisher in data['Publishers']:
   publishers[publisher] = ex_url('publisher',publisher)

developers = {}
for developer in data['Developers']:
   developers[developer] = ex_url('developer',developer)

graphic_designer_name = data['Graphic Designer'][0]
graphic_designer_uri = ex_url('graphic_designer', graphic_designer_name)
graphic_designer = {graphic_designer_name: graphic_designer_uri}

awards = {}
for award in data['Awards']:
   awards[award] = ex_url('award',award)


#creacion de nodos

g.add((game_uri, RDF.type, EX["BoardGame"]))
g.add((game_uri, EX["name"], Literal("Lost Ruins of Arnak")))

# relacion anio juego
g.add((game_uri, EX['anio_salida'], Literal(data['Year Released'][0], datatype=XSD.gYear)))

# relacion nombres alternativos juego
for nombres_alternativos in data['Alternate Names']:
    g.add((game_uri, EX["nombres_alternativos"], Literal(nombres_alternativos)))
    g.add((game_uri, RDF.type, EX["Alternate_Names"]))
    g.add((game_uri, EX["name"], Literal(nombres_alternativos)))

# relacion categorias juego
for categorias in data['Categories']:
    g.add((game_uri, EX["Category"], Literal(categorias)))
    g.add((game_uri, RDF.type, EX["Category"]))
    g.add((game_uri, EX["name"], Literal(categorias)))

# relacion mecanicas juego
for mechanism in data['Mechanisms']:
    g.add((game_uri, EX["Mechanics_de_juegos"], Literal(mechanism)))
    g.add((game_uri, RDF.type, EX["Mechanics"]))
    g.add((game_uri, EX["name"], Literal(mechanism)))

# relacion familia juego
for familia_de_juegos in data['Family']:
    g.add((game_uri, EX["familia_de_juegos"], Literal(familia_de_juegos)))
    g.add((game_uri, RDF.type, EX["Family"]))
    g.add((game_uri, EX["name"], Literal(familia_de_juegos)))


# relacion publishers juego
for publisher_name, publisher_uri in publishers.items():
    g.add((game_uri, EX["Publisher_juego"], publisher_uri))
    g.add((publisher_uri, RDF.type, EX["Publisher"]))
    g.add((publisher_uri, EX["name"], Literal(publisher_name)))

# relacion designers juego
for designer_name, designer_uri in designers.items():
    g.add((game_uri, EX["diseniador_juego"], designer_uri))
    g.add((designer_uri, RDF.type, EX["Designer"]))
    g.add((designer_uri, EX["name"], Literal(designer_name)))

# relacion artistas juego
for artist_name, artist_uri in artists.items():
    g.add((game_uri, EX["artista_juego"], artist_uri))
    g.add((artist_uri, RDF.type, EX["Artist"]))
    g.add((artist_uri, EX["name"], Literal(artist_name)))

# relacion developer juego
for developer_name, developer_uri in developers.items():
    g.add((game_uri, EX["developer_juego"], developer_uri))
    g.add((developer_uri, RDF.type, EX["Developer"]))
    g.add((developer_uri, EX["name"], Literal(developer_name)))

# relacion graphic designer juego
for graphic_designer_name, graphic_designer_uri in graphic_designer.items():
    g.add((game_uri, EX["graphic_designer_juego"], graphic_designer_uri))
    g.add((graphic_designer_uri, RDF.type, EX["Graphic_Designer"]))
    g.add((graphic_designer_uri, EX["name"], Literal(graphic_designer_name)))


# relacion premios juego
for award_name, award_uri in awards.items():
    g.add((game_uri, EX["award_juego"], award_uri))
    g.add((award_uri, RDF.type, EX["Award"]))
    g.add((award_uri, EX["name"], Literal(award_name)))

# realcion deisgners casados
for designer1_name, designer1_uri in designers.items():
    for designer2_name, designer2_uri in designers.items():
        if designer1_name != designer2_name:
            g.add((designer1_uri, EX["casado_con"], designer2_uri))


# relacion entre dewsigners, artistas, developers y graphic designer
all_entities = {**designers, **artists, **developers, **graphic_designer}

# Crear relaciones "trabaja_con" entre todos los nodos
for entity1_uri in all_entities.values():
    for entity2_uri in all_entities.values():
        if entity1_uri != entity2_uri:
            g.add((entity1_uri, EX["trabaja_con"], entity2_uri))


# relacion premios con deisgners
for designer_uri in designers.values():
    for award_uri in awards.values():
        g.add((award_uri, EX["premio_con"], designer_uri))


g.serialize("boardgame_graph.rdf", format="turtle")

<Graph identifier=N864c912d789741adaeda4d4dded38af7 (<class 'rdflib.graph.Graph'>)>

## Base de datos tabulares

In [18]:
# Se buscan los archivos que terminen en CSV, de la carpeta que anteriormente se descargó desde Google Drive

carpeta = 'lost-ruins-of-arnak'
archivos_csv = []
for archivo_pdf in os.listdir(carpeta):
    if archivo_pdf.endswith('.csv'):
        archivos_csv.append(os.path.join(carpeta, archivo_pdf))

# Con el mismo caso que los archivos PDF, ordenamos los archivos por nombre, para que no haya problemas al realizarle las operaciones y limpiezas
archivos_csv = sorted(archivos_csv)

# Se cambia el nombre de la columna ítem, y se modifican distintos nombres de ítems que, entre los 2 archivos, son un poco diferentes
# Lo que genera que no se puede realizar al operación del merge
# Por último se pasan todas las variables numéricas a str para facilitar luego el proceso a la LLM

herramientas_stats = pd.read_csv(archivos_csv[0])
herramientas_stats.drop(columns=['Image'], inplace=True)
herramientas_numerico = pd.read_csv(archivos_csv[1])
herramientas_numerico_tool = herramientas_numerico.rename(columns={'Tool': 'Item'})
herramientas_numerico_tool['Item'] = herramientas_numerico_tool['Item'].replace({'Bow & Arrows': 'Bow and Arrows', 'Air Drop': 'Airdrop','Carrier Pigeon': 'Carrier Pidgeon'})
herramientas_numerico_tool.drop(columns=['Cost'], inplace = True)
df = pd.merge(herramientas_stats, herramientas_numerico_tool, on='Item', how='inner')
df.drop(columns=['Notes'], inplace=True)
df['Difference'] = df['Difference'].astype(str)
df['Cost'] = df['Cost'].astype(str)
df['Benefit'] = df['Benefit'].astype(str)
df['Recursive Inflation'] = df['Recursive Inflation'].astype(str)
df['Versatility'] = df['Versatility'].astype(str)

## Clasificadores

### Clasificador por ejemplos

No es necesario ejecutar esta celda de código, ya que se eligió utilizar la LLM, como modelo de clasificación

In [19]:
# # Se prueba y califica el clasificador, por ejemplo, usando un dataset hecho a mano sobre pregunta de los 3 tipos de base de datos

# url_entrenamiento = 'https://github.com/Nicolas-Noir/NLP-TUIA-2024/raw/main/TP2/Archivos/datos_entrenamiento/querys_entrenamiento.csv'

# querys_entrenamiento = pd.read_csv(url_entrenamiento)

# len(querys_entrenamiento)

# x_train, x_test, y_train, y_test = train_test_split(querys_entrenamiento['pregunta'], querys_entrenamiento['clase'], test_size=0.25, random_state=11)

# x_train = x_train.reset_index(drop=True)
# x_test = x_test.reset_index(drop=True)

# x_train_vectorized = model_multi.encode(x_train)
# x_test_vectorized = model_multi.encode(x_test)

# modelo_LR = LogisticRegression(max_iter=3000, solver='lbfgs')
# modelo_LR.fit(x_train_vectorized, y_train)

# y_pred_LR = modelo_LR.predict(x_test_vectorized)
# acc_LR = accuracy_score(y_test, y_pred_LR)
# report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

# print("Precisión Regresión Logística:", acc_LR)
# print("Reporte de clasificación Regresión Logística:\n", report_LR)

### Clasificador LLM

In [20]:
def ask(prompt: str):
    client = InferenceClient(api_key=api_key_hf)

    # Se le pasa a la LLM, el contexto de los nombres de las columnas de Dataset, además de los nombres principales de los nodos y el nombre de los archivos de los documentos
    columns = list(df.columns)
    graph_keys = list(data.keys())
    descripciones = list({descripcion for tuplas in lista_embedding_chunks for descripcion, _, _ in tuplas})

    messages = [
        {
        "role": "system",
        "content": (f"""
                You are an AI assistant expert in the board game The Lost Ruins of Arnack
                Your task in this case is to accurately classify the question to be entered into 3 categories, such as opinions/rules, relationships or statistics.
                The classification will depend on the topics to which the questions are directed:
                #in the case of opinions/rules, it is expected that they will be questions of a more general index about the board game, such as: how many players can play the game, what are the rules, what are the advantages of some item."
                #In the case of relationships, it is expected that they will be related to the designers, or artists who created the game, also the year it was developed, or the different names they have in other languages ​​or general relationships with the board game
                #In the case of statistics, it is about numerical values ​​of the different items of the game, such as speed, efficiency, cost, best use and other attributes. Or items with certain values ​​or uses
                Only respond with one of the 3 categories mentioned, classifying intelligently
                """)
        },
        {
            "role": "user",
            "content": f"""classify the following question into one of the following classes: opinions/rules, relationships, statistics.

            Respond ONLY with the following words: opinions/rules, relationships, statistics.

            Data context:
            - text data titles: {descripciones}
            - Tabular data columns: {columns}
            - Graph data keys: {graph_keys}

            Examples:
            What strategies are most effective for winning? - opinions/rules
            What are the adventajes of the cart Tent? -  opinions/rules
            Who are the designers involved in this game? - relationships
            what mechanics has the game? - relationships
            which are the alternative names for the game? - relationships
            In what year the game was released? - relationships (this is the only numeric data in the relationships)
            What is the value of the Tent card? - statistics
            which is best use for the cart pickace? - statistics
            What items have efficiency greater than 300%? - statistics
            the game can be played alone? - opinions/rules

            If the question does not clearly fit into any category, choose 'opinions/rules'.

            Here is the question to classify:
            {prompt} -
            """
        }
    ]

    completion = client.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=messages,
        max_tokens=100
    )

    return completion.choices[0].message.content


## LLM

### LLM datos tabulares

In [21]:
def datos_tabulares(query: str):
    client = InferenceClient(api_key=api_key_hf)

    # Se pasan las columnas del Dataset, y la primera fila, para que tenga contexto de qué tipo de datos hay en el Dataset
    columns = list(df.columns)
    first_row = df.iloc[0].to_dict()

    messages = [
        {
        "role": "system",
        "content": (
                    "You are an AI assistant that answers user querys using only Pandas language"
                    "Use the information and context provided to you"
                    "Your responses should be concise, precise, and directly address the query, answering only what is asked, without adding anything beyond the Pandas query."
                    )
        },
        {
            "role": "user",
            "content": f"""
    I have a Pandas DataFrame with the following columns: {columns}.

    Here is an example of the first row of the DataFrame:
    {first_row}

    I want you to write a query in pandas to answer the following question:
    '{query}'.

    Return ONLY the QUERY in PANDAS, without explanation or additional text.

    Instructions:
    1. Use `.loc` to filter rows and select specific columns.
    2. Don't use double indexing like `df[df[...]`.
    3. Make sure you handle pandas methods like `.gt`, `.any`, and `.loc` correctly.
    4. If numerical operations are required and the columns contain strings, convert them to numeric values using `pd.to_numeric()` or `astype(float)` before applying conditions.
    5. Return only what is requested; for example, if the question is about how many items have a value greater than 1, return only the items.
    6. Return ONLY the QUERY in PANDAS, without explanation or additional text.
    7. Make sure the code is clean, functional, and does not include characters like \\n at the beginning or end or with ''' at the beginning or end.
    """
        }
    ]

    completion = client.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=messages,
        max_tokens=200
    )

    return list(eval(completion.choices[0].message.content))


### LLM busqueda de grafos

In [22]:
def buscar_grafos(query:str):
  client = InferenceClient(api_key=api_key_hf)

  # Se pasa las relaciones del grafo en formato turtle, para que tenga un contexto y entienda cómo está formado el grafo
  messages = [
      {
      "role": "system",
      "content": (
            "You are an AI assistant that answers user queries using only sparql language"
            "Use the information and context provided to you"
            "Your responses should be concise, precise, and directly address the query, answering only what is asked, without adding anything more than the sparql query."
                )
    },
    {
      "role": "user",
      "content": f""" I have a RDF graph database with the following format: {g.serialize(format="turtle")}.
    I want you to write a query in RDF to answer the following question:
    '{query}'.

    Return me only the query in RDF, without explanation or additional text.
    Make sure you only return clean, functional code, without characters like \\n at the beginning or end or thinbks like that ```sparql\n at the beginning or end.
    """
    }
  ]

  completion = client.chat.completions.create( model="Qwen/Qwen2.5-72B-Instruct", messages=messages, max_tokens=200)
  text_context = []


  consulta_query = completion.choices[0].message.content

  consulta = g.query(consulta_query)

  for row in consulta:
    text_context.append(row[0])

  nombres_publicadores = [str(uri).split('/')[-1] for uri in text_context]
  return nombres_publicadores


### Busqueda hibrida + ReRanker

In [23]:
result = collection.get()
result['documents']

def consulta_vectorial_hibrida(query: str):

    # Codificamos las querys y buscamos dentro de chromadb, los chunks que mayor cercanía tengan a los embeddings de los mismos
    def consulta_vectorial_embedding(query: str):
        respuestas = []
        pregunta_query = model_multi.encode(query)

        results = collection.query(query_embeddings=[pregunta_query], n_results=3)
        for resultados in results['documents'][0]:
            respuestas.append(resultados)

        return respuestas

    # Realiza una consulta vectorial basada en palabras clave utilizando el modelo BM25.
    def consulta_vectorial_key_word(query_kw: str):
        chunks = []
        token_chunk = []
        lista_chunks_key_word = []

        # Tokenización de los documentos y creación del índice BM25
        for chunk in result['documents']:
          token_chunk.append(word_tokenize(chunk))
          chunks.append(chunk)

        bm25 = BM25Okapi(token_chunk)
        query_token = word_tokenize(query_kw)

        # Calcula los puntajes de similitud entre la consulta y los documentos.
        scores = bm25.get_scores(query_token)
        top_indices = np.argsort(scores)[-3:][::-1]

        for inx in top_indices:
            lista_chunks_key_word.append(chunks[inx])
        return lista_chunks_key_word

    # Obtener los chunks de ambas consultas
    chunks_vectorial = consulta_vectorial_key_word(query)
    chunks_embedding = consulta_vectorial_embedding(query)

    # Fusionar y eliminar duplicados
    all_chunks = list(set(chunks_vectorial + chunks_embedding))

    # Crear la lista de pares para el reranker
    lista_chunks_scores = [[query, chunk] for chunk in all_chunks]

    # Calcular las puntuaciones con el reranker
    puntuaciones = reranker.compute_score(lista_chunks_scores, normalize=True)

    # Encontrar el chunk con la mejor puntuación
    max_index = np.argmax(puntuaciones)
    mejor_chunk = all_chunks[max_index]


    return mejor_chunk


### LLM contexto

In [24]:
def llm_consultas(query: str):

  def llm_query_contexto(query: str, chunk: str) -> str:
    client = InferenceClient(api_key=api_key_hf)

    messages = [
        {
            "role": "system",
            "content": (f"""
                #You are an expert AI assistant in the board game The Lost Ruins of Arnack
                #You must answer the question that is going to be introduced to you, using the context.
                #You can receive 3 types of contexts:
                *One with extensive information about the given question.
                *Another with names of people, year of release of the board game, or alternative game names.
                *Another about numerical values, according to the questions to the items of the board game The Lost Ruins of Arnack, where you will only be given context, a number, or a list with several names of the items.
                #Your answers must be to directly address the query, answering broadly, or only answering with the context directly in the cases where you are given a numerical value or the list of items.
                #Try to improve the format of the answer, with the context that is provided to you as an intelligent and expert AI.
            """)
        },
        {
            "role": "user",
            "content": (f"""
                  With the following question '{query}'. and the following context {chunk}.
                  #try to give a coherent answer as an expert AI in board games, even if the context is simple or just a number.
                  #Return only the answer to the question, using mostly the context that is provided to you.
                  #Try to be organized and concise when answering the question

                  Examples

                  #Question: What is the speed of the hat
                  #context: 0

                  you can answer this by saying that: The speed of the hat is 0
                  #That answer is accurate and sufficient for your task as an expert AI in board games

                    """)
          }
        ]

    completion = client.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=messages,
        max_tokens=500
    )

    return completion.choices[0].message.content


  # En caso de que la pregunta de entrada sea en inglés, no se traducirá nada, ya que las instrucciones y los archivos están configurados y guardados en inglés
  # En el caso de que esté en español, se traduce la pregunta, usando Google Translate, al inglés, y luego se traduce la respuesta de la LLM, devuelta al español
  # En el caso de que falle la base de datos tabulares o de grafos, se prueba una segunda vez, pero con la vectorial

  idioma = 'en'

  if detect(query) != 'en':
    idioma = 'es'
    query = GoogleTranslator(source='es', target='en').translate(query)


  try:
    if ask(query) == 'opinions/rules':
      contexto_vectorial = consulta_vectorial_hibrida(query)
      query_vectorial = llm_query_contexto(query, contexto_vectorial)
      if idioma == 'en':
        return query_vectorial

      else:
        query_espanol_vectorial = GoogleTranslator(source='en', target='es').translate(query_vectorial)
        return query_espanol_vectorial


    elif ask(query) == 'relationships':
      contexto_grafos = buscar_grafos(query)
      query_grafos = llm_query_contexto(query, contexto_grafos)
      if idioma == 'en':
        return query_grafos

      else:
        query_espanol = GoogleTranslator(source='en', target='es').translate(query_grafos)
        return query_espanol


    else:
      contexto_tabular = datos_tabulares(query)

      query_pandas = llm_query_contexto(query, contexto_tabular)
      if idioma == 'en':
        return query_pandas
      else:
        query_espanol_tabular = GoogleTranslator(source='en', target='es').translate(query_pandas)
        return query_espanol_tabular


  except Exception as e:
    print(f"Error: {e}. Intentando con el proceso de 'opinions/rules'...\n")

    # En caso de error en 'relationships' o en el else, se intenta 'opinions/rules'
    if ask(query) != 'opinions/rules':
      query_vectorial_exepcion = llm_query_contexto(query, consulta_vectorial_hibrida(query))
      if idioma == 'en':
        return query_vectorial_exepcion

      else:
        query_espanol_vectorial_exepcion = GoogleTranslator(source='en', target='es').translate(query_vectorial_exepcion)
        return query_espanol_vectorial_exepcion



#### Chat

In [25]:
def chat_preguntas(pregunta):

    print(llm_consultas(pregunta))

print("\nBienvenido al chat. Puedes hacer tus preguntas o presiona Enter para salir.")

while True:
    pregunta = input("\nEscribe tu pregunta: ")

    if not pregunta.strip():
        print("\nSaliendo del chat. ¡Hasta luego!")
        break

    chat_preguntas(pregunta)

    print("\nPuedes hacer tus preguntas o presiona Enter para salir.")



Bienvenido al chat. Puedes hacer tus preguntas o presiona Enter para salir.

Escribe tu pregunta: What are the principal rules of the game?


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


The principal rules of The Lost Ruks of Arnack are as follows:

1. **Draw**: All players draw from their decks until they have a hand of 5 Cards. If a player has insufficient Cards in the deck to fill their hand, they draw all they can.
2. **Take Turns**: Beginning with the Starting Player and proceeding clockwise, players take turns. On their turn, they perform 1 Main Action and any number of Unlimited Actions. The Main Actions are:
   - **Dig at a Site**
   - **Discover a New Site**
   - **Overcome a Guardian**
   - **Buy a Card**
   - **Play a Card**
   - **Research**
   - **Pass**
3. **Pass**: If a player has completed all actions they wish to perform on their turn, they may Pass. Once all players have passed, proceed to the next step.
4. **Set Up for Next Round**:
   - Shuffle all the cards in your play area (not your deck) and return them to the bottom of your deck.
   - Set the Game Board up for the next round.
   - Pass the Starting Player Marker to the left.
5. **Move the Moon

#### Ejemplos de querys al RAG

In [26]:
#'What items have Cost greater than 1?'
#'¿Qué ítems tienen Costo mayor a 1?'

#'in what year the game was released?'
#'¿cuales son los nombres de los diseñadores?'

#'What are the principal rules of the game?'
#'¿Cuantos jugadores pueden jugar el juego?'

# Parte 2 / Creacion de Agente

### Instalaciones

In [27]:
#Descarga de Ollama
!curl -fsSL https://ollama.com/install.sh | sh

#Iniciamos Ollama en background
!rm -f ollama_start.sh
!echo '#!/bin/bash' > ollama_start.sh
!echo 'ollama serve' >> ollama_start.sh
#Make the script executable
!chmod +x ollama_start.sh
!nohup ./ollama_start.sh &

!ollama pull llama3.2 > ollama.log

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  p

### Configuración del logger

In [28]:
import time
import logging

# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

### Funciones

In [29]:
def wikipedia_search(query: str) -> str:
  try:
    wikipedia.set_lang('en')
    return wikipedia.summary(query, sentences=3)

  except Exception as e:
    return f"No se encontró información en wikipedia por el siguiente error: {e}"


def doc_search(query: str) -> str:
  return consulta_vectorial_hibrida(query)


def graph_search(query: str) -> list[str]:
  return buscar_grafos(query)


def table_search(query: str) -> str:
  return datos_tabulares(query)



tools = [
    FunctionTool.from_defaults(fn=wikipedia_search, description="Search for information on Wikipedia. Use: search text"),
    FunctionTool.from_defaults(fn=doc_search, description="Search for information in the documents saved in the database, providing any type of information related to the game. Use: search text"),
    FunctionTool.from_defaults(fn=graph_search, description="Search for information related to the relationship of different artists, and people who made the game, in addition to its mechanics and similar properties. Use: Simplified and simple search text"),
    FunctionTool.from_defaults(fn=table_search, description="Obtains numerical data or items about the different attributes of the items, whether percentages or numeric. Use: search text"),
]



### Carga de modelo

In [30]:
modelo_llm = Ollama(
    model="llama3.2:latest",

    request_timeout=4000.0,
    temperature=0.1,
    context_window=4096
)

Settings.LLM = modelo_llm
!nohup litellm --model ollama/llama3.2:latest --port 8000 > litellm.log 2>&1 &


### Instrucciones

In [31]:
agent = ReActAgent.from_tools(
    tools,
    llm=modelo_llm,
    verbose=True,
    chat_formatter=ReActChatFormatter(),
    system_prompt=

    """
    1. **Analyze the user's query**: Identify whether it is a numeric, textual, or relational query. Determine whether a tool is needed to answer it.

    2. **Use the correct format for each tool's input**:
    - **Valid JSON format**: Make sure to use double quotes ("") for keys and values.
    - Examples for table_search:
    - Efficiency query:
    Action Input: {"query": "efficiency > 300"}
    - Price query:
    Action Input: {"query": "price < 50"}

    3. **Error handling**:
    - If a tool returns an error, check:
    1. Is the input format valid?
    2. Is the query defined correctly?
    - Only retry if you can correct the problem (e.g., by using double quotes or adjusting the syntax).

    4. **Avoid unnecessary iterations**:
    - Stop the process if the same error occurs more than two consecutive times.
    - Use an iteration limit to prevent infinite loops.

    5. **Provide a clear final answer**:
    - Combine all valid observations obtained from the tools.
    - Give a complete and well-structured answer to the user.

    6. **Format examples for tools**:
    - **doc_search**: Action Input: "text to find" / use it as principal tool
    - **wikipedia_search**: Action Input: "topic"
    - **graph_search**: Action Input: "relationships or names"
    - **table_search**: Action Input: {"query": "numeric or tabular data"}

    7. **Context for queries on tables**:
    - The table contains columns such as: "Item", "Efficiency", "Cost", "Category".
    - "Efficiency" refers to the output/input ratio of a process.

    8. **Validate inputs before running them**:
    - If the input contains filters or complex parameters, structure them in a valid JSON format.
    - Corrected input example:
    Action Input: {"query": "items with efficiency greater than 300"}

    9. **General interaction format**:
    - Always use the following format:
    Thought: [Reasoning]
    Action: tool_name
    Action Input: [Input in the correct format]
    Observation: [Tool output]
    Final Answer: [Complete final answer]

    10. **Testing and validation** (optional):
    - Run tests to ensure that inputs are handled correctly. For example:

python
    def test_table_search():
    input = {"query": "efficiency > 300"}
    assert is_valid_json(input), "Input is not valid JSON"
    result = run_tool("table_search", input)
    assert "Error" not in result, "Error in table_search"

    """
)

#### Chat

In [32]:
def chat_con_agente(query: str):
    """
    Función para interactuar con el agente ReAct.
    """
    try:
        if not query.strip():
            return "La consulta está vacía"

        response = agent.chat(query)
        logger.info(f"Consulta: {query}")
        logger.info(f"Respuesta: {response}")
        return response
    except Exception as e:
        return f"Error al procesar la consulta: {str(e)}"

def ejecutar_ejemplo():
    print("=== Ejemplo de interacción con el agente ReAct ===")

    query = 'who are the names of the designers of the game?'

    response = chat_con_agente(query)
    print(f"Respuesta: {response}")


if __name__ == "__main__":
    # Configurar el entorno de ejecución
    import logging
    logging.basicConfig(level=logging.INFO)

    # Ejecutar el ejemplo
    ejecutar_ejemplo()

=== Ejemplo de interacción con el agente ReAct ===
> Running step f2c7c80e-499d-4495-ba0b-f2836811ddfe. Step input: who are the names of the designers of the game?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: graph_search
Action Input: {'query': 'designers of the game'}
Observation: ['Elwen', 'Mín']
> Running step 1dfd725f-f925-46d9-ad9a-269ffe759f42. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The designers of the game are Elwen and Mín.
Respuesta: The designers of the game are Elwen and Mín.
